## 1. Import Data

In [1]:
import pickle
import gzip
import numpy as np
import os
import cv2

In [2]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [3]:
def save_zipped_pickle(obj, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, 2)

In [4]:
def evaluate(predictions, targets):
    ious = []
    for p, t in zip(predictions, targets):
#         assert p['name'] == t['name']
#         prediction = np.array(p['prediction'], dtype=bool)
#         target = np.array(t['label'], dtype=bool)

        assert targets.shape == predictions.shape
        overlap = predictions*targets
        union = predictions + targets

        ious.append(overlap.sum()/float(union.sum()))
    
#     print("Median IOU: ", np.median(ious))
    return(np.median(ious))
# evaluate(y_pred,y_test)

## 1.1 Load Data

In [5]:
# load data
train_data = load_zipped_pickle("./task3/train.pkl")
test_data = load_zipped_pickle("./task3/test.pkl")
samples = load_zipped_pickle("./task3/sample.pkl")



数据结构：  
    train_data[1-65]包含65个病人的数据, 前46个是amateur数据， 后19个是expert数据  
    train_data[1]['video','label','dataset'...] , video是图片数组， label是MV mask，dataset代表是amateur还是expert  
    train_data[1]['video'] shape: [IMG_WIDTH, IMG_HEIGHT, FRAME NUM]， label也一样

In [6]:
# test_img = train_data[60]['label'][:,:,24]
# zero_img = np.zeros(test_img.shape)
# zero_img[test_img] = 255
# zero_img_resize = cv2.resize(zero_img, (1000,1000))
# zero_img.shape

In [7]:
# cv2.imshow("mask", zero_img_resize)
  
# cv2.waitKey(0) 

# cv2.destroyAllWindows() 

## 1.2 Data pre-processing

In [8]:
# 样本图片大小
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 1

In [9]:
# 找到含有标记的帧
def is_labeled(mask_list):
    
    label_index = []
    for i in range((mask_list).shape[2]):
        if(True in mask_list[:,:,i]):
            label_index.append(i)
            continue
    return np.array(label_index)

def image_size_normalize(raw_data, size):
    #选出有标记的图片并resize到相应尺寸
    img_width, img_height, img_channel = size
    
    labeled_img_list = []
    mask_list = []
    
    for patient_i in range(len(raw_data)):

        img_i = raw_data[patient_i]['video'] # Shape: [IMG_WIDTH1, IMG_HEIGHT1, Frame_Num1]
        mask_i = raw_data[patient_i]['label']
        label_index = is_labeled(mask_i)
        labeled_img_i = img_i[:,:,label_index]
        labeled_mask_i = mask_i[:,:,label_index]
        
        for frame_j in range(labeled_mask_i.shape[2]):
            img_j = labeled_img_i[:,:,frame_j] # [IMG_WIDTH2, IMG_HEIGHT2, Frame_Num2]
            mask_j = labeled_mask_i[:,:,frame_j]
            mask_j_digi = np.zeros(mask_j.shape)
            mask_j_digi[mask_j] = 1
            if(img_j.shape[0]!= IMG_WIDTH or img_j.shape[1]!= IMG_HEIGHT):
                img_j = cv2.resize(img_j,(img_width, img_height))
                mask_j_digi = cv2.resize(mask_j_digi,(img_width, img_height))
            labeled_img_list.append(img_j)
            mask_list.append(mask_j_digi)
            
    return np.array(labeled_img_list).reshape(-1,img_width, img_height, img_channel),np.array(mask_list).reshape(-1,img_width, img_height, 1)

In [10]:
# 从训练集中选出所有label过的图像并resize到指定大小
img_labeled, mask = image_size_normalize(train_data, (IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS))  # OutPut Shape: [Labeled_Frame_Num, IMG_WIDTH2, IMG_HEIGHT2, IMG_CHANNEL]

In [11]:
img_labeled.shape

(195, 256, 256, 1)

## 2 U-Net

## 2.1Model Definition 

In [12]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import random

/Users/zdh/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/zdh/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/zdh/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/zdh/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [13]:
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

In [14]:
#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 256, 256, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 160         lambda[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 256, 256, 16) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (

## 2.2 Train

In [15]:
results = model.fit(img_labeled, mask, batch_size=10, epochs=10)

Epoch 1/10
195/195 [==============================] - 94s 484ms/step - loss: 0.2661 - acc: 0.9236
Epoch 2/10
195/195 [==============================] - 84s 433ms/step - loss: 0.1433 - acc: 0.9883
Epoch 3/10
195/195 [==============================] - 90s 461ms/step - loss: 0.1433 - acc: 0.9883
Epoch 4/10
195/195 [==============================] - 87s 446ms/step - loss: 0.1433 - acc: 0.9883
Epoch 5/10
195/195 [==============================] - 86s 443ms/step - loss: 0.1433 - acc: 0.9883
Epoch 6/10
195/195 [==============================] - 83s 427ms/step - loss: 0.1433 - acc: 0.9883
Epoch 7/10
195/195 [==============================] - 87s 444ms/step - loss: 0.1433 - acc: 0.9883
Epoch 8/10
195/195 [==============================] - 88s 453ms/step - loss: 0.1433 - acc: 0.9883
Epoch 9/10
195/195 [==============================] - 90s 463ms/step - loss: 0.1433 - acc: 0.9883
Epoch 10/10
195/195 [==============================] - 84s 433ms/step - loss: 0.1433 - acc: 0.9883


## cross validation

In [ ]:
from sklearn.model_selection import KFold
import sklearn
scores = []
kf = KFold(n_splits=5, shuffle=True)
x_select = img_labeled
y_select = mask
for train_idx, test_idx in kf.split(x_select):
    x_train = x_select[train_idx]
    y_train = y_select[train_idx]
    x_test = x_select[test_idx]
    y_test = y_select[test_idx]
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
#     s = r2_score(y_test.ravel(), y_pred.ravel())
    s= evaluate(y_pred,y_test)
    print('score: ', s)
    scores.append(s)

scores

Epoch 1/1
156/156 [==============================] - 77s 491ms/step - loss: 0.1462 - acc: 0.9881
score:  0.0
Epoch 1/1
156/156 [==============================] - 85s 542ms/step - loss: 0.1409 - acc: 0.9884
score:  0.0
Epoch 1/1
128/156 [=======================>......] - ETA: 21s - loss: 0.1446 - acc: 0.9881

In [ ]:
def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

print(jaccard_similarity(y_test.ravel(),y_pred.ravel()))

In [ ]:
def correlation(a,b):
    set_a = set(a)
    set_b = set(b)
    unions = len(set_a.union(set_b))
    intersections = len(set_a.intersection(set_b))
    return 1. * intersections / unions
correlation(y_test.ravel(),y_pred.ravel())

In [ ]:
def evaluate(predictions, targets):
    ious = []
    for p, t in zip(predictions, targets):
#         assert p['name'] == t['name']
#         prediction = np.array(p['prediction'], dtype=bool)
#         target = np.array(t['label'], dtype=bool)

        assert targets.shape == predictions.shape
        overlap = predictions*targets
        union = predictions + targets

        ious.append(overlap.sum()/float(union.sum()))
    
#     print("Median IOU: ", np.median(ious))
    return(np.median(ious))
# evaluate(y_pred,y_test)

In [ ]:
import pandas as pd
pd.DataFrame(y_test.flatten()).to_csv("C:/Users/Junling_W/Desktop/y_test",index = False)

In [ ]:

pd.DataFrame(mask.flatten()).to_csv("C:/Users/Junling_W/Desktop/mask.csv",index = False)

In [ ]:

pd.DataFrame(y_pred.flatten()).to_csv("C:/Users/Junling_W/Desktop/y_pred",index = False)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
s = r2_score(y_test.ravel(), y_pred.ravel())

In [ ]:
s

## 2.3 MODEL PREDICT

In [ ]:
# 从训练集中选出所有label过的图像并resize到指定大小
img_labeled= image_size_normalize(test_data, (IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS))  # OutPut Shape: [Labeled_Frame_Num, IMG_WIDTH2, IMG_HEIGHT2, IMG_CHANNEL]

In [ ]:
model.fit(img_labeled, mask, batch_size=10, epochs=10)

# 3 Visualization

In [ ]:
import os

from PIL import Image

def video_make(name, image_list, mask_list,  img_size =(112, 112)):

    fps = 24 #视频每秒24帧
    size = img_size #需要转为视频的图片的尺寸

    #视频保存在当前目录下

    fourcc = cv2.VideoWriter_fourcc(*'MJPG')    
    video = cv2.VideoWriter(name+".avi",fourcc, fps,size, False)
    #draw stuff that goes on every frame here
    for frame_i in range(image_list.shape[2]):
        img = (image_list[:,:,frame_i])
        mask = mask_list[:,:,frame_i]
        img[mask] = 255
        img_mat = cv2.Mat(img)
        video.write(img)
    video.release()


## 4 Conclusion（things to be done for improvement）  
4.1 如上可见，随着epoch增加, accuracy波动很大，网络不是很work。 不清楚是数据量不够，网络复杂度不够， 还是iter比较小（纯ml小白）  
    如果是数据量不够， 或许需要用tracking的办法增加数据量
4.2 需要做cross-validation的预测版本
4.3 由于数据集中， 属于MV的像素点很少，和类别不平衡问题很像， 最好能增大 MV区域误识别的错误权重
4.4 预测的可视化实现